In [133]:
import requests
import re
from urllib.request import urlopen
from bs4 import BeautifulSoup
import json
import time
import pandas as pd
import numpy as np

In [106]:
year, month, day, team = 2018, 10, 19, 'ORL'
web_template = (f'https://www.basketball-reference.com/boxscores/{year}{month}{day}0{team}.html')

In [127]:
def web_scrape():
    year, month, day, team = 2018, 10, 19, 'ORL'    
    web_template = (f'https://www.basketball-reference.com/boxscores/{year}{month}{day}0{team}.html')
    data = requests.get(web_template)
    soup = BeautifulSoup(data.text, 'html.parser')
    headers_four_factors = [th.getText() for th in soup.findAll('tr', limit=2)[1].findAll('th')]
    rows = soup.findAll('tr')[2:]
    player_stats1 = [[td.getText() for td in rows[i].findAll('td')] for i in range(len(rows))]
    player_names1 = [[td.getText() for td in rows[i].findAll('th')] for i in range(len(rows))]
    stats = pd.DataFrame(player_stats1, columns = headers_four_factors[1:])
    player = pd.DataFrame(player_names1)
    player = player[0][:66]
    stats['Player'] = player
    return stats

In [128]:
def team_summary():
    year, month, day, team = 2018, 10, 19, 'ORL'    
    web_template = (f'https://www.basketball-reference.com/boxscores/{year}{month}{day}0{team}.html')
    data = requests.get(web_template)
    soup = BeautifulSoup(data.text, 'html.parser')
    rows2 = soup.findAll(class_='scorebox')
    overall_teams = [strong.getText() for strong in rows2[0].findAll('strong')]
    overall_teams = [items.strip('\n') for items in overall_teams]
    overall_score = [scores.getText() for scores in rows2[0].findAll(class_='scores')]
    overall_score = [items.strip('\n') for items in overall_score]
    
    def date_adjustment():
        overall_date = [dates.getText() for dates in rows2[0].findAll(class_='scorebox_meta')]
        overall_date = [items.strip('\n') for items in overall_date]
        overall_date_2 = [items.split(',') for items in overall_date]
        output_list = []
        output_list.append(overall_date_2[0][0])
        output_list.append(overall_date_2[0][1])
        output_list.append(overall_date_2[0][2][:5].strip(' '))
        
        return output_list
    
    
    date_list = [' '.join(date_adjustment())] * 2
    teams_scores = pd.DataFrame(overall_teams, columns=['Team_Name'])
    teams_scores['Score'] = overall_score
    teams_scores['Date'] = date_list
    return teams_scores
    

In [129]:
web_template

'https://www.basketball-reference.com/boxscores/201810190ORL.html'

In [134]:
test = web_scrape()

In [135]:
team_summary()

,Team_Name,Score,Date
0,Charlotte Hornets,120,7:00 PM October 19 2018
1,Orlando Magic,88,7:00 PM October 19 2018


In [136]:
test

,MP,FG,FGA,FG%,3P,3PA,3P%,FT,FTA,FT%,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,+/-,Player
0,29:59,6,8,.750,3,4,.750,0,0,,...,6,6,5,2,0,2,0,15,+32,Nicolas Batum
1,26:46,8,16,.500,5,10,.500,5,5,1.000,...,2,2,5,1,0,3,0,26,+34,Kemba Walker
2,25:05,2,7,.286,0,2,.000,4,4,1.000,...,6,7,1,1,1,0,0,8,+19,Jeremy Lamb
3,22:45,3,8,.375,0,0,,2,2,1.000,...,5,8,2,1,0,0,2,8,+17,Cody Zeller
4,20:29,3,7,.429,2,6,.333,0,0,,...,3,4,0,0,0,1,0,8,+13,Marvin Williams
5,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,Reserves
6,23:37,4,12,.333,2,6,.333,1,1,1.000,...,1,1,2,0,0,1,3,11,+5,Malik Monk
7,22:29,5,8,.625,0,2,.000,2,4,.500,...,7,9,5,0,2,1,1,12,+24,Michael Kidd-Gilchrist
8,17:18,2,5,.400,1,1,1.000,1,2,.500,...,2,5,2,1,0,2,4,6,+15,Willy Hernangómez
9,16:12,0,5,.000,0,1,.000,0,0,,...,3,3,6,0,0,2,2,0,+3,Tony Parker


In [137]:
def four_factors_output():
    four_factors_dataframe_test = stats[['Player','MP','FG', 'FGA', '3P', 'FT', 'ORB', 'TOV', 'FTA', 'DRB']]
    #     four_factors_dataframe_test['FG'] = pd.to_numeric(four_factors_dataframe_test['FG'], errors='coerce').fillna(0).astype(int)
    #     four_factors_dataframe_test['FGA'] = pd.to_numeric(four_factors_dataframe_test['FGA'], errors='coerce').fillna(0).astype(int)
    #     four_factors_dataframe_test['3P'] = pd.to_numeric(four_factors_dataframe_test['3P'], errors='coerce').fillna(0).astype(int)
    #     four_factors_dataframe_test['TOV'] = pd.to_numeric(four_factors_dataframe_test['TOV'], errors='coerce').fillna(0).astype(int)
    #     four_factors_dataframe_test['FTA'] = pd.to_numeric(four_factors_dataframe_test['FTA'], errors='coerce').fillna(0).astype(int)
    #     four_factors_dataframe_test['FT'] = pd.to_numeric(four_factors_dataframe_test['FT'], errors='coerce').fillna(0).astype(int)
    #     four_factors_dataframe_test['ORB'] = pd.to_numeric(four_factors_dataframe_test['ORB'], errors='coerce').fillna(0).astype(int)
    #     four_factors_dataframe_test['DRB'] = pd.to_numeric(four_factors_dataframe_test['DRB'], errors='coerce').fillna(0).astype(int)
    for items in test_list:
        four_factors_dataframe_test[items] = pd.to_numeric(four_factors_dataframe_test[items], errors='coerce').fillna(0).astype(int)
    
    
    pass
